In [1]:
import cvxpy as cp
import cvxopt
import numpy as np
import clarabel
import mosek
import xpress
import cylp
import cplex
import sdpap
import pyscipopt


### Data

In [2]:
a=[0.5, -0.5, 0.2, -0.7, 0.6, -0.2, 0.7, -0.5, 0.8, -0.4]
l=[40, 20, 40, 40, 20, 40, 30, 40, 30, 60]
Preq=np.arange(a[0],a[0]*(l[0]+0.5),a[0])
for i in range(1, len(l)):
    Preq=np.r_[ Preq, np.arange(Preq[-1]+a[i],Preq[-1]+a[i]*(l[i]+0.5),a[i]) ]

T = sum(l)

Peng_max = 20.0
Pmg_min = -6.0
Pmg_max = 6.0
eta = 0.1
gamma = 0.1

### Car with battery

In [3]:
Ebatt_max = 100.0
Peng = cp.Variable(T)
Pmg = cp.Variable(T)
Pbr = cp.Variable(T)
Ebatt = cp.Variable(T+1)

In [4]:
constraints = []
obj = 0

constraints += [Ebatt[T] == Ebatt[0]]
for t in range(T):
    constraints += ([Preq[t] == Peng[t] + Pmg[t] - Pbr[t], 0 <= Peng[t], Peng[t] <= Peng_max, Pmg_min <= Pmg[t], Pmg[t] <= Pmg_max, Pbr[t] >= 0, 
                    0 <= Ebatt[t], Ebatt[t] <= Ebatt_max, eta*Pmg[t] >= -Ebatt[t] + Pmg[t] + Ebatt[t+1],
                    eta*Pmg[t] <= -Ebatt[t+1] - Pmg[t] +Ebatt[t]])
    
    obj += cp.sum(Peng[t] + gamma*(Peng[t]**2))

In [5]:
prob = cp.Problem(cp.Minimize(obj), constraints)
status = prob.solve()
print(f"task 1: {prob.value}")

task 1: 5077.525371239874


### Car with no battery

In [6]:
Ebatt_max = 0.0
Peng = cp.Variable(T)
Pmg = cp.Variable(T)
Pbr = cp.Variable(T)
Ebatt = cp.Variable(T+1)

In [7]:
constraints = []
obj = 0

constraints += [Ebatt[T] == Ebatt[0]]
for t in range(T):
    constraints += ([Preq[t] == Peng[t] + Pmg[t] - Pbr[t], 0 <= Peng[t], Peng[t] <= Peng_max, Pmg_min <= Pmg[t], Pmg[t] <= Pmg_max, Pbr[t] >= 0, 
                    0 <= Ebatt[t], Ebatt[t] <= Ebatt_max, eta*Pmg[t] >= -Ebatt[t] + Pmg[t] + Ebatt[t+1],
                    eta*Pmg[t] <= -Ebatt[t+1] - Pmg[t] +Ebatt[t]])
    
    obj += cp.sum(Peng[t] + gamma*(Peng[t]**2))

In [8]:
prob = cp.Problem(cp.Minimize(obj), constraints)
status = prob.solve()
print(f"task 2: {prob.value}")

task 2: 5896.808990170339


### Version with general solving function

In [9]:
a=[0.5, -0.5, 0.2, -0.7, 0.6, -0.2, 0.7, -0.5, 0.8, -0.4]
l=[40, 20, 40, 40, 20, 40, 30, 40, 30, 60]
Preq=np.arange(a[0],a[0]*(l[0]+0.5),a[0])
for i in range(1, len(l)):
    Preq=np.r_[ Preq, np.arange(Preq[-1]+a[i],Preq[-1]+a[i]*(l[i]+0.5),a[i]) ]

T = sum(l)

Peng_max = 20.0
Pmg_min = -6.0
Pmg_max = 6.0
eta = 0.1
gamma = 0.1
Ebatt_max = 100.0

In [69]:
def general_solver(E_max):

    Peng = cp.Variable(T)
    Pmg = cp.Variable(T)
    Pbr = cp.Variable(T)
    Ebatt = cp.Variable(T+1)

    constraints = []
    obj = 0

    constraints += [Ebatt[T] == Ebatt[0]]
    for t in range(T):
        constraints += ([Preq[t] == Peng[t] + Pmg[t] - Pbr[t], 0 <= Peng[t], Peng[t] <= Peng_max, Pmg_min <= Pmg[t], Pmg[t] <= Pmg_max, Pbr[t] >= 0, 
                        0 <= Ebatt[t], Ebatt[t] <= E_max, eta*Pmg[t] >= -Ebatt[t] + Pmg[t] + Ebatt[t+1],
                        eta*Pmg[t] <= -Ebatt[t+1] - Pmg[t] +Ebatt[t]])
        
        obj += cp.sum(Peng[t] + gamma*(Peng[t]**2))

    prob = cp.Problem(cp.Minimize(obj), constraints)
    status = prob.solve(solver="OSQP", eps_rel=1e-7, eps_abs=1e-7)
    retval = {}
    retval["Peng"] = list(Peng.value)
    retval["Pmg"] = list(Pmg.value)
    retval["Pbr"] = list(Pbr.value)
    retval["E"] = list(Ebatt.value)
    return retval, prob.objective.value

def general_solver_glitch(E_max):

    Peng = cp.Variable(T)
    Pmg = cp.Variable(T)
    Pbr = cp.Variable(T)
    Ebatt = cp.Variable(T+1)
    eps = 1e-5


    constraints = []
    obj = 0

    constraints += [Ebatt[T] == Ebatt[0]]
    for t in range(T):
        constraints += ([Preq[t] == Peng[t] + Pmg[t] - Pbr[t], 0 <= Peng[t], Peng[t] <= Peng_max, Pmg_min <= Pmg[t], Pmg[t] <= Pmg_max, Pbr[t] >= 0, 
                        0 <= Ebatt[t], Ebatt[t] <= E_max, eta*Pmg[t] >= -Ebatt[t] + Pmg[t] + Ebatt[t+1],
                        eta*Pmg[t] <= -Ebatt[t+1] - Pmg[t] +Ebatt[t]])
        
        obj += cp.sum(Peng[t] + gamma*(Peng[t]**2) + eps*cp.maximum(0, -Pmg[t]))

    prob = cp.Problem(cp.Minimize(obj), constraints)
    status = prob.solve(solver='OSQP', eps_rel=1e-7, eps_abs=1e-7)
    retval = {}
    retval["Peng"] = list(Peng.value)
    retval["Pmg"] = list(Pmg.value)
    retval["Pbr"] = list(Pbr.value)
    retval["E"] = list(Ebatt.value)

    return retval, prob.objective.value
    

In [11]:
# print(f"Task 1: {general_solver(Ebatt_max)}", f"Task 2: {general_solver(0)}", sep="\n")

### Handling glitches

In [60]:
def general_solver_glitch(E_max):

    Peng = cp.Variable(T)
    Pmg = cp.Variable(T)
    Pbr = cp.Variable(T)
    Ebatt = cp.Variable(T+1)
    eps = 1e-3


    constraints = []
    obj = 0

    constraints += [Ebatt[T] == Ebatt[0]]
    for t in range(T):
        constraints += ([Preq[t] == Peng[t] + Pmg[t] - Pbr[t], 0 <= Peng[t], Peng[t] <= Peng_max, Pmg_min <= Pmg[t], Pmg[t] <= Pmg_max, Pbr[t] >= 0, 
                        0 <= Ebatt[t], Ebatt[t] <= E_max, eta*Pmg[t] >= -Ebatt[t] + Pmg[t] + Ebatt[t+1],
                        eta*Pmg[t] <= -Ebatt[t+1] - Pmg[t] +Ebatt[t]])
        
        obj += cp.sum(Peng[t] + gamma*(Peng[t]**2) + eps*cp.maximum(0, -Pmg[t]))

    prob = cp.Problem(cp.Minimize(obj), constraints)
    status = prob.solve(solver='OSQP', eps_abs=1e-6, eps_rel=1e-6)
    print(prob.objective.value)
    retval = {}
    retval["Peng"] = list(Peng.value)
    retval["Pmg"] = list(Pmg.value)
    retval["Pbr"] = list(Pbr.value)
    retval["E"] = list(Ebatt.value)

    return retval